In [42]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import cv2
from pycocotools.coco import COCO
from PIL import Image
import os
import matplotlib.pyplot as plt
#import colormap as cmap
import numpy as np
import torch

def visualize(index, masks, bounding_boxes):
    image = coco.loadImgs(coco.getImgIds()[index])[0]    
    anno_ids = coco.getAnnIds(imgIds=[image["id"]])
    annos = coco.loadAnns(anno_ids) 
    
    """new_annos = []
    for single in annos:
        single = get_angles(single)
        new_annos.append([single])
    
    annos = new_annos"""
    filename = os.path.join(data_folder,image["file_name"])
    img = cv2.imread(str(filename))
    print(img.shape)
    for anno in annos:
        #print(anno)
        anno = get_angles(anno)
        #c = tuple([int(x) for x in anno["category_id"]])         
        if masks:
            points = np.array(anno["segmentation"], dtype=np.int32).reshape(1,-1,2)
            img = cv2.polylines(img, points, True, 3)
        
        if bounding_boxes:  
                x,y,w,h,a = np.array(anno["bbox"]).astype(np.int32)
                #img = cv2.rectangle(img, (x,y),(x + w,y + h),(0,255,255), 3) 
                img = draw_xywha(img,x,y,w,h,a)
                cat = coco.cats[anno['category_id']]['name']
                cv2.putText(img, cat, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX ,  0.8, 1, cv2.LINE_AA)            
                
    
    plt.imshow(img[...,::-1])
    plt.axis("off")
    plt.grid(False)
    plt.tight_layout()

def get_angles(ann):
    
    #print(ann['bbox'])
    img_id = ann['image_id']
    new_ann = None
    # get width and height
    if not 'rbbox' in ann:
        # using COCO dataset. 4 = [x1,y1,w,h]
        coco = True
        # convert COCO format: x1,y1,w,h to x,y,w,h
        ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
        ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
        ann['bbox'].append(0)
        if ann['bbox'][2] > ann['bbox'][3]:
            ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
            ann['bbox'][4] -= 90
        new_ann = ann['bbox']
    else:
        # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
        assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
        new_ann = ann['rbbox']
        

        if new_ann[2] > new_ann[3]:
            new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
            new_ann[4] -= 90 if new_ann[4] > 0 else -90

    if new_ann[2] == new_ann[3]:
        new_ann[3] += 1 # force that w < h
        
    new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

    assert new_ann[2] < new_ann[3]
    assert new_ann[4] >= -90 and new_ann[4] < 90

    # override original bounding box with rotated bounding box
    ann['bbox'] = torch.Tensor(new_ann)
    #print(ann['bbox'])
    return ann

    
def draw_xywha(im, x, y, w, h, angle, color=(255,0,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w/2, -h/2], [w/2, -h/2], [w/2, h/2], [-w/2, h/2]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([x, y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    return im
    
    
# data_folder = "/localdata/saurabh/dataset/FES/JPEGImages/"
# coco = COCO("/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json")
data_folder = "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/JPEGImages/"
coco = COCO("/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/instances_training.json")
plt.figure(figsize=(100,100))
total = len(coco.getImgIds()) - 1

interact(visualize, index=widgets.IntSlider(min=0, max=total, step=1, continuous_update=False), masks=False, bounding_boxes=True);  
plt.show()

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


<Figure size 7200x7200 with 0 Axes>

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=300), Checkbox(valu…

# Make the data files

In [6]:
import glob
from collections import defaultdict
import os
import json
import cv2
image_paths = glob.glob("data/test/dst_images/*")

image_ids = []
image_names = []
for image in image_paths:
    image_name = os.path.splitext(os.path.basename(image))[0]
    print(image_name)
    image_names += [image_name]
    image_name = image_name.split('_')[1]
    image_ids += [image_name]
    
    
print(image_ids)

Record_00803
Record_00804
Record_00802
Record_00800
Record_00801
['00803', '00804', '00802', '00800', '00801']


In [3]:
image_names

['Record_00803',
 'Record_00804',
 'Record_00802',
 'Record_00800',
 'Record_00801']

In [4]:
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                # convert COCO format: x1,y1,w,h to x,y,w,h
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2], ann['bbox'][3] = ann['bbox'][3], ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                # x,y,w,h,a
                assert len(ann['rbbox']) == 5, 'Unknown bbox format'
                new_ann = ann['rbbox']

                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3], new_ann[2]
                    new_ann[4] += 90 - np.finfo(np.float32).eps

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1  # force that w < h

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [16]:
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

load_anns("data/test/images", "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/labels_training.json" )

Loading annotations /localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/labels_training.json into memory...


In [8]:
image_ids = []
for i in range(len(imgid2info)):
    file_name = imgid2info[i+1]['file_name'].split('.')[0]
    if file_name in image_names:
        image_ids += [i+1]

In [9]:
image_ids

[23, 57, 207, 218, 237]

In [10]:
ind_path = [imgid2path[ids] for ids in image_ids]
annotations = [imgid2anns[ids] for ids in image_ids]

In [11]:
label_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5}

In [12]:
label_map[3]

2

In [13]:
all_images = []
for i, ann in enumerate(annotations):
    file_name = image_names[i]
    #text_file = open('/localdata/saurabh/yolov3/data/test/dst_labels/{}.txt'.format(file_name), "w")
    for anno in ann:
        print(anno['category_id'])
        text_file.write("%f " %label_map[anno['category_id']])
        text_file.writelines(["%f " % item  for item in anno['bbox']])
        text_file.write("\n")
    
    text_file.close()
    print(file_name)
    


Record_00803
Record_00804
Record_00802
Record_00800
Record_00801


In [15]:
annotations

[[], [], [], [], []]

In [31]:
with open('/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/captions_training.json', 'r') as f:
            json_data = json.load(f)

In [34]:
json_data['images']

[{'id': 1,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00722.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 2,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00670.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 3,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00810.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 4,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00782.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 5,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00636.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 6,
  'width': 1680,
  'height': 1680,
  'file_name': 'Record_00783.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 {'id': 7,
  'width': 1680,
  'height': 1680,
  'fil